<h1> Book Recommender <h1>
<h3> The focus of this project is to build a simple and straightforward Book Recommender using similirty as our driver for suggestions </h3>

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

import warnings
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

In [3]:
book_rating_path = "/workspaces/ProjectPortfolio/1. Data Science/2. KNN - Book Recommender/Datasets/BX-Book-Ratings.csv"
book_path = "/workspaces/ProjectPortfolio/1. Data Science/2. KNN - Book Recommender/Datasets/BX-Books.csv"
book_user_path = "/workspaces/ProjectPortfolio/1. Data Science/2. KNN - Book Recommender/Datasets/BX-Users.csv"


book_rating_df = pd.read_csv(
    book_rating_path,
    encoding = "ISO-8859-1",
    sep=";")

book_df = pd.read_csv(
    book_path,
    encoding = "ISO-8859-1",
    sep=";",
    on_bad_lines="skip")

book_user_df = pd.read_csv(
    book_user_path,
    encoding = "ISO-8859-1",
    sep=";")

In [4]:
book_rating_df.head()


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [5]:
book_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [6]:
book_user_df.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [7]:
book_merged_df = pd.merge(book_rating_df, book_user_df, on='User-ID', how='inner')
book_merged_df = pd.merge(book_merged_df, book_df, on='ISBN', how='inner')

book_merged_df.head()

,User-ID,ISBN,Book-Rating,Location,Age,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,"tyler, texas, usa",NaN,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,2313,034545104X,5,"cincinnati, ohio, usa",23.0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
2,6543,034545104X,0,"strafford, missouri, usa",34.0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
3,8680,034545104X,5,"st. charles county, missouri, usa",2.0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
4,10314,034545104X,9,"beaverton, oregon, usa",NaN,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...


In [8]:
book_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1021327 entries, 0 to 1021326
Data columns (total 12 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   User-ID              1021327 non-null  int64  
 1   ISBN                 1021327 non-null  object 
 2   Book-Rating          1021327 non-null  int64  
 3   Location             1021327 non-null  object 
 4   Age                  745773 non-null   float64
 5   Book-Title           1021327 non-null  object 
 6   Book-Author          1021325 non-null  object 
 7   Year-Of-Publication  1021327 non-null  object 
 8   Publisher            1021325 non-null  object 
 9   Image-URL-S          1021327 non-null  object 
 10  Image-URL-M          1021327 non-null  object 
 11  Image-URL-L          1021323 non-null  object 
dtypes: float64(1), int64(2), object(9)
memory usage: 93.5+ MB


In [9]:
book_merged_df.describe(include='all')

,User-ID,ISBN,Book-Rating,Location,Age,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
count,1.021327e+06,1021327,1.021327e+06,1021327,745773.000000,1021327,1021325,1021327.0,1021325,1021327,1021327,1021323
unique,NaN,260564,NaN,22412,NaN,232612,98344,202.0,16260,260272,260272,260269
top,NaN,0971880107,NaN,"toronto, ontario, canada",NaN,Wild Animus,Stephen King,2002.0,Ballantine Books,http://images.amazon.com/images/P/0971880107.0...,http://images.amazon.com/images/P/0971880107.0...,http://images.amazon.com/images/P/0971880107.0...
freq,NaN,2502,NaN,14722,NaN,2502,10044,86798.0,34630,2502,2502,2502
mean,1.397008e+05,NaN,2.843067e+00,NaN,37.377943,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,8.038459e+04,NaN,3.855029e+00,NaN,14.094138,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,2.000000e+00,NaN,0.000000e+00,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,6.974450e+04,NaN,0.000000e+00,NaN,28.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,1.394670e+05,NaN,0.000000e+00,NaN,35.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,2.093730e+05,NaN,7.000000e+00,NaN,45.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
book_merged_df.isna().sum()

User-ID                     0
ISBN                        0
Book-Rating                 0
Location                    0
Age                    275554
Book-Title                  0
Book-Author                 2
Year-Of-Publication         0
Publisher                   2
Image-URL-S                 0
Image-URL-M                 0
Image-URL-L                 4
dtype: int64

In [11]:
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [14]:
columns_to_drop = ['ISBN', 'User-ID','Image-URL-S','Image-URL-M','Image-URL-L']
book_merged_df_filtered = book_merged_df.drop(columns=columns_to_drop)

columns_to_check = ['Publisher', 'Book-Author']

# Drop rows with null values in the specified columns
book_merged_df_filtered.dropna(subset=columns_to_check, inplace=True)

# Reset the index after dropping rows
book_merged_df_filtered.reset_index(drop=True, inplace=True)

imputer = SimpleImputer(strategy='median')
book_merged_df_filtered[['Age']] = imputer.fit_transform(book_merged_df_filtered[['Age']])

book_merged_df_filtered['Year-Of-Publication'].astype('float')

ValueError: could not convert string to float: 'DK Publishing Inc'

In [ ]:
book_merged_df_filtered.info()

NameError: name 'book_merged_df_filtered' is not defined

In [ ]:
book_merged_df_filtered.isna().sum()

Book-Rating            0
Location               0
Age                    0
Book-Title             0
Book-Author            0
Year-Of-Publication    0
Publisher              0
dtype: int64